# Use this notbook for your 3 choices of APIs

In [ ]:
import requests
import json
import pandas as pd
from getpass import getpass

#### Using Edamam API for Recipes search

In [ ]:
edamam_base_url = 'https://api.edamam.com/api/recipes/v2'
edamam_app_id = getpass(prompt='Enter Edamam APP ID: ')
edamam_app_key = getpass(prompt='Enter Edamam APP KEY: ')

In [ ]:
def extract_recipes_with_ingredients(hit):
    return {
        'recipe': hit['recipe']['label'],
        'ingredients': extract_ingredients_names(hit['recipe']['ingredients'])
    }


def extract_ingredients_names(ingredients):
    return [ingredient['food'] for ingredient in ingredients]


def get_recipes(search_term):
    response = requests.get(f'{edamam_base_url}?type=public&q={search_term}&app_id={edamam_app_id}&app_key={edamam_app_key}')
    if response.status_code != 200:
        print(f"""Error: {response.text}""")
    else:
        try:
            return [extract_recipes_with_ingredients(hit) for hit in response.json()['hits']]
        except:
            print(f"Could not parse results of '{search_term}'")

In [ ]:
recipes = get_recipes('chicken')
len(recipes)

In [ ]:
recipes

#### Using Spoonacular APIs to get random Recipes

In [ ]:
spoonacular_base_url = 'https://api.spoonacular.com'
spoonacular_api_key = getpass(prompt='Enter Spoonacular API KEY: ')

In [ ]:
def extract_recipe_with_ingredients(recipe):
    return {
        'recipe': recipe['title'],
        'ingredients': extract_ingredient_names(recipe['extendedIngredients'])
    }


def extract_ingredient_names(ingredients):
    return [ingredient['nameClean'] for ingredient in ingredients]


def get_random_recipes(nb_recipes):
    response = requests.get(f'{spoonacular_base_url}/recipes/random?apiKey={spoonacular_api_key}&number={nb_recipes}')
    if response.status_code != 200:
        print(f"""Error: {response.text}""")
    else:
        try:
            return [extract_recipe_with_ingredients(recipe) for recipe in response.json()['recipes']]
        except:
            print('Could not parse results.')
            return []

In [ ]:
random_recipes = get_random_recipes(3)

In [ ]:
random_recipes

#### Using Open Food Facts to get available products from recipes in a particular supermarket

In [ ]:
openfoodfacts_base_url = 'https://world.openfoodfacts.org'

In [ ]:
def get_available_organic_products(country, supermarket, ingredient):
    response = requests.get(f'{openfoodfacts_base_url}/api/v2/search?categories_tags={ingredient}&countries_tags_en={country}&labels_tags=organic&stores_tags={supermarket}&fields=product_name')
    if response.status_code != 200:
        print(f"""Error: {response.text}""")
    else:
        try:
            return [product['product_name'] for product in response.json()['products'] if 'product_name' in product]
        except:
            print('Could not parse results.')
            return []

In [ ]:
def fetch_available_organic_products(country, supermarket, recipes):
    for recipe in recipes:
        enriched_ingredients = []
        for ingredient in recipe['ingredients']:
            products = get_available_organic_products(country, supermarket, ingredient)
            enriched_ingredients.append({
                'name' : ingredient,
                'products_in_supermarket': products
            })
        recipe['ingredients'] = enriched_ingredients
    
    return recipes

In [ ]:
random_recipes = fetch_available_organic_products('germany', 'rewe', random_recipes)

In [ ]:
random_recipes

#### Use Case

In [ ]:
# I want to have random recipes for my week

week_recipes = get_random_recipes(7)

In [ ]:
# I check if it is possible to get all ingredients in order to make these dishes

week_recipes = fetch_available_organic_products('germany', 'rewe', week_recipes)
len(week_recipes)

In [ ]:
# I filter recipes that are not doable

doable_recipes = []
for recipe in week_recipes:
    doable = True
    for ingredient in recipe['ingredients']:
        if ingredient['products_in_supermarket'] == []:
            doable = False
            break
    if doable == True:
        doable_recipes.append(recipe)

len(doable_recipes)

In [ ]:
# Apparently these recipes are not doable because we cannot buy all ingredients. We get recipes from ingredients we have then

recipes_from_ingredients = get_recipes('chicken,egg,soy sauce,noodles')[:7]
len(recipes_from_ingredients)

In [ ]:
recipes_from_ingredients